In [1]:
"""
We use following lines because we are running on Google Colab
If you are running notebook on a local computer, you don't need this cell
"""
from google.colab import drive
drive.mount('/content/gdrive')
import os
os.chdir('/content/gdrive/My Drive/finch/scrapy/')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
!pip install scrapy

In [3]:
import scrapy
from scrapy.crawler import CrawlerProcess


class CarSpider(scrapy.Spider):
    name = 'car'
    
    allowed_domains = 'autohome.com.cn'
    
    start_urls = [
        'http://www.autohome.com.cn/grade/carhtml/A.html',
        'http://www.autohome.com.cn/grade/carhtml/B.html'
    ]
    
    custom_settings = {
      'FEED_FORMAT':'csv',
      'FEED_URI': 'car.csv'
    }

    
    def parse(self, response):
      for level_1_idx in range(len(response.xpath('//dl').extract())):
        level_1_idx = str(level_1_idx + 1)
        level_1_item = response.xpath('//dl[' + level_1_idx + ']/dt/div[1]/a/text()').extract()[0]
        level_2_items = response.xpath('//dl[' + level_1_idx + ']/dd//div[@class="h3-tit"]/a/text()').extract()

        for level_2_idx, level_2_item in enumerate(level_2_items):
          level_2_idx = str(level_2_idx + 1)
          level_3_items = response.xpath('//dl[' + level_1_idx + ']/dd//ul[@class="rank-list-ul"][' + level_2_idx + ']/li[@id]').extract()

          for level_3_idx in range(len(level_3_items)):
            level_3_idx = str(level_3_idx + 1)

            for car_name in response.xpath('//dl[' + level_1_idx + ']/dd//ul[@class="rank-list-ul"][' + level_2_idx + ']/li[@id][' + level_3_idx + ']/h4/a/text()').extract():
              car_price = response.xpath('//dl[' + level_1_idx + ']/dd//ul[@class="rank-list-ul"][' + level_2_idx + ']/li[@id][' + level_3_idx + ']/div[1]/a/text()').extract()[0]
              min_price, max_price = self.process_price(car_price)
              
              yield {
                  '品牌': level_1_item,
                  '子品牌': level_2_item,
                  '车名': car_name,
                  '最低价': min_price,
                  '最高价': max_price
              }
    
    
    def process_price(self, price):
      prices = price.split('-')
      if len(prices) != 2:
          max_price = '暂无'
          min_price = '暂无'
      else:
          max_price = str(prices[1].replace('万', ''))
          min_price = str(prices[0])
      return min_price, max_price
                

process = CrawlerProcess()
process.crawl(CarSpider)
process.start()

2019-10-16 09:07:27 [scrapy.utils.log] INFO: Scrapy 1.7.3 started (bot: scrapybot)
2019-10-16 09:07:27 [scrapy.utils.log] INFO: Versions: lxml 4.2.6.0, libxml2 2.9.8, cssselect 1.1.0, parsel 1.5.2, w3lib 1.21.0, Twisted 19.7.0, Python 3.6.8 (default, Jan 14 2019, 11:02:34) - [GCC 8.0.1 20180414 (experimental) [trunk revision 259383]], pyOpenSSL 19.0.0 (OpenSSL 1.1.1c  28 May 2019), cryptography 2.7, Platform Linux-4.14.137+-x86_64-with-Ubuntu-18.04-bionic
2019-10-16 09:07:27 [scrapy.crawler] INFO: Overridden settings: {'FEED_FORMAT': 'csv', 'FEED_URI': 'car.csv'}
2019-10-16 09:07:27 [scrapy.extensions.telnet] INFO: Telnet Password: c6092071dc916115
2019-10-16 09:07:27 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.memusage.MemoryUsage',
 'scrapy.extensions.feedexport.FeedExporter',
 'scrapy.extensions.logstats.LogStats']
2019-10-16 09:07:27 [scrapy.middleware] INFO: Enabled downloade